<a href="https://colab.research.google.com/github/h-gokul/HHAR-Using_HARnet/blob/master/HHAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from scipy.stats import skew, kurtosis
from scipy.signal import hilbert
from multiprocessing import Pool, cpu_count
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dropout, Lambda
from keras.layers import BatchNormalization, UpSampling1D, UpSampling2D, LSTM, merge, Concatenate,Reshape

from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler
import pickle
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
# from scipy.misc import imresize #deprecated
import scipy.signal
from scipy.signal import cwt, decimate
import pywt
import os
#from keras_squeezenet import SqueezeNet
# from SqueezeNetModel import SqueezeNet
from keras import regularizers
from keras import backend as K

In [0]:
import os
from google.colab import drive 
drive.mount ("/content/gdrive")
os.chdir("/content/gdrive/My Drive/HHAR")
cwd = os.getcwd()   
# print the current directory 
print("Current working directory is:", cwd) 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#DataExtraction based on sampling rates

In [0]:
df1= pd.read_csv("Phones_accelerometer.csv")
df1.columns

Index(['Index', 'Arrival_Time', 'Creation_Time', 'x', 'y', 'z', 'User',
       'Model', 'Device', 'gt'],
      dtype='object')

In [0]:
#nan_removal
print(df1['gt'].value_counts())
print(df1['gt'].unique())
df_=pd.DataFrame()
df_= df_.append(df1[df1['gt']== 'stand'])
df_= df_.append(df1[df1['gt']== 'sit'])
df_= df_.append(df1[df1['gt']== 'walk'])
df_= df_.append(df1[df1['gt']== 'bike'])
df_= df_.append(df1[df1['gt']== 'stairsup'])
df_= df_.append(df1[df1['gt']== 'stairsdown'])
print(df_['gt'].value_counts())
print(df_['gt'].unique())

walk          2192401
sit           1991919
stand         1851492
bike          1845557
stairsup      1782010
stairsdown    1615896
Name: gt, dtype: int64
['stand' nan 'sit' 'walk' 'stairsup' 'stairsdown' 'bike']
walk          2192401
sit           1991919
stand         1851492
bike          1845557
stairsup      1782010
stairsdown    1615896
Name: gt, dtype: int64
['stand' 'sit' 'walk' 'bike' 'stairsup' 'stairsdown']


In [0]:
# del df1
# Create dictionaries for activities , devices and users.
label_ = {}
dev_ = {}
user_ = {}

df_ = df_[df_['gt'] != 'null'] 

acts=df_['gt'].unique()
print(acts)
for i, a in enumerate(acts):
  label_[a]= i  
print(label_) #dictionary for 6 activity classes

devices= df_['Device'].unique()
print(devices)
for i, dev in enumerate(devices):
  dev_[dev]= i  
print(dev_) #dictionary for 8 devices

users= df_['User'].unique()
print(users)
for i, user in enumerate(users):
  user_[user]= i  
print(user_) #dictionary for 9 users

['stand' 'sit' 'walk' 'bike' 'stairsup' 'stairsdown']
{'stand': 0, 'sit': 1, 'walk': 2, 'bike': 3, 'stairsup': 4, 'stairsdown': 5}
['nexus4_1' 'nexus4_2' 's3_1' 's3_2' 's3mini_1' 's3mini_2' 'samsungold_1'
 'samsungold_2']
{'nexus4_1': 0, 'nexus4_2': 1, 's3_1': 2, 's3_2': 3, 's3mini_1': 4, 's3mini_2': 5, 'samsungold_1': 6, 'samsungold_2': 7}
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i']
{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8}


In [0]:
#to see shape
stand = df_[df_['gt']=='stand'] ## in the entire dataframe, choose the stand class.
s32=stand[stand["Device"]=='s3_2'] ## inside the stand class, select  s3_2 device among others. 
user_a = s32[s32["User"]=='a'] ## inside s3_2 device, there is many users, we see user a 
print(user_a.shape) ## print the shape of user a in s3_2 in stand class.

testArray_a = np.array(user_a) #indexed as [row][column]
np.shape(testArray_a)

(28144, 10)


(28144, 10)

In [0]:
print(testArray_a[0])
print(df_.columns)

[0 1424696633914 274248797337000 -6.0525417 1.0821792 7.374141000000002
 'a' 's3' 's3_2' 'stand']
Index(['Index', 'Arrival_Time', 'Creation_Time', 'x', 'y', 'z', 'User',
       'Model', 'Device', 'gt'],
      dtype='object')


In [0]:
def MinWindowFn(device): ##chooses a the window size for downsampling(ie decimation) depending on device. 
    if str(device) == 'nexus4_1' or str(device) == 'nexus4_2':
      min_w = 400
    elif str(device) == 's3_1' or str(device) == 's3_2':
      min_w = 300
    elif str(device) == 's3mini_1' or str(device) == 's3mini_2':
      min_w = 200
    else:
      min_w = 100
    return min_w
  
def ExtractWindow(df,w): 
  window_count= df.shape[0]//w ## crops out a divisible portion from the available stream of accelerometer data. say if df.shape= 91k, w=400, window_count = 227 windows    
  n=window_count*w ## n =  no. of windows * w 
  df = df.iloc[0:n]   ## extract the divisible portion of the dataframe.
  return df

d_400=pd.DataFrame()
d_300=pd.DataFrame()
d_200=pd.DataFrame()
d_100=pd.DataFrame()
acctemp, devtemp, actstemp = [],[],[]
for a in acts:
  df_acts = df_[df_['gt']== a]
  for d in devices:
    df_dev = df_acts[df_acts['Device']== d ]
    for u in users:
      df_user = df_dev[df_dev['User'] == u]
      df_user.drop(columns=["Index",'Arrival_Time', 'Creation_Time',"Model"], axis=1 , inplace=True)
      wlen = MinWindowFn(d)
      if (df_user.shape[0]>wlen): 
        if wlen==400 : 
          d_400 =d_400.append( ExtractWindow(df_user,wlen))               
        elif wlen==300 : 
          d_300 = d_300.append( ExtractWindow(df_user,wlen))
        elif wlen==200 : 
          d_200 = d_200.append( ExtractWindow(df_user,wlen))
        else : 
          d_100 = d_100.append( ExtractWindow(df_user,wlen))
  print(f'{a} is Done')
print ("All data are arranged Devices and label wise.")


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


stand is Done
sit is Done
walk is Done
bike is Done
stairsup is Done
stairsdown is Done
All data are arranged Devices and label wise.


In [0]:
## print the total number of windows of data is available per user
print((d_400['User'].value_counts()/400))
print(d_300['User'].value_counts()/300)
print(d_200['User'].value_counts()/200)
print(d_100['User'].value_counts()/100)

path = os.getcwd() +'/'
d_400.to_csv(path+'d_400.csv',index=False)
d_300.to_csv(path+'d_300.csv',index=False)
d_200.to_csv(path+'d_200.csv',index=False)
d_100.to_csv(path+'d_100.csv',index=False)

i    1594.0
e    1566.0
f    1560.0
b    1543.0
d    1481.0
g    1479.0
a    1445.0
h    1411.0
c    1381.0
Name: User, dtype: float64
i    1052.0
b    1043.0
e    1033.0
g    1018.0
a     970.0
h     966.0
d     960.0
c     938.0
f     506.0
Name: User, dtype: float64
f    862.0
i    825.0
g    814.0
h    800.0
b    777.0
e    775.0
a    743.0
c    729.0
d    717.0
Name: User, dtype: float64
b    2284.0
f    2201.0
i    2184.0
e    2136.0
g    2131.0
h    2041.0
c    2022.0
a    1945.0
d    1943.0
Name: User, dtype: float64


# Decimation and Reshape

There are unequal sampling rates ranging from 200Hz to 50 Hz depending on the devices used. So these were already segregated based on Sampling rates and now the signals need to be decimated to 50Hz on a preprocessing stage. 


In [0]:
#Run this to  see the data shape

stand = d_300[d_300['gt']=='stand'] ## in the entire dataframe, choose the stand class.
s32=stand[stand["Device"]=='s3_1'] ## inside the stand class, select  s3_2 device among others. 
user_a = s32[s32["User"]=='f'] ## inside s3_2 device, there is many users, we see user a 
print(user_a.shape) ## print the shape of user a in s3_2 in stand class.
testArray_a = np.array(user_a) #indexed as [row][column]
np.shape(testArray_a)

In [0]:
os.chdir("/content/gdrive/My Drive/HHAR")
d_400 =  pd.read_csv('d_400.csv')
d_300 =  pd.read_csv('d_300.csv')
d_200 =  pd.read_csv('d_200.csv')
d_100 =  pd.read_csv('d_100.csv')

In [0]:
from collections import Counter

def Decimate(df,w):
  x_ = []
  y_ = []
  count_tot,count_d,count_a = 0,0,0
  wlen = w/100
  acts = (df['gt'].unique())
  devs = (df['Device'].unique())
  user = (df['User'].unique())
  for a in acts:
    df_act = df[df["gt"]==a]
    count_a=count_a+1
    for d in devs:
      df_dev = df_act[df_act["Device"]==d]
      count_d=count_d+1
      count_u=0
      for u in user:
        df_user = df_dev[df_dev['User']==u]
        count_tot=count_tot+1
        count_u=count_u+1
        if (df_user.shape[0]> wlen):
          x =  list(decimate(np.array(df_user['x']),int(wlen)))
          y =  list(decimate(np.array(df_user['y']),int(wlen)))
          z =  list(decimate(np.array(df_user['z']),int(wlen)))  
          ulabel = np.repeat(u,len(x)) 
          dlabel = np.repeat(d,len(x))
          alabel = np.repeat(a,len(x))
          x_temp = (np.vstack((x,y,z))).transpose()
          x_temp = x_temp.reshape(-1,100,3) # reshaping to a window
          y_temp = np.vstack((alabel,ulabel,dlabel)).transpose()
          y_temp = y_temp.reshape(-1,100,3)
          y_temp = y_temp[:,0,:] #creating corresponding labels
          if (count_tot == 1):
            x_ = x_temp
            y_ = y_temp
          else:
            x_ = np.vstack((x_,x_temp))
            y_ = np.vstack((y_,y_temp))
  return (np.array(x_),np.array(y_)) 


In [0]:
#All are decimated to equal sampling rates of 50 Hz
x_400,y_400 = Decimate(d_400,400)
x_300,y_300 = Decimate(d_300,300)
x_200,y_200 = Decimate(d_200,200)
x_100,y_100 = Decimate(d_100,100)

x = np.vstack((x_400,x_300,x_200,x_100))
y = np.vstack((y_400,y_300,y_200,y_100))

In [0]:
print(x.shape,y[:,0].shape) # Final X and Y of the dataset

(47875, 100, 3) (47875,)


# Discrete Wavelet Transformation of the Signals
